In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoConfig, PhobertTokenizer
import numpy as np
import os
import time
import re
import sys
sys.path.append('/home3/phungqv/post_correction/vietnam_number')
from vietnam_number import w2n, w2n_single
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [2]:
tokenizer = PhobertTokenizer.from_pretrained('/home3/phungqv/post_correction/phobert_tokenizer')
# pretrained = AutoModelForSeq2SeqLM.from_pretrained('/home3/phungqv/data_generate/text-normalization/checkpoint-84561')
pretrained = AutoModelForSeq2SeqLM.from_pretrained('/home3/phungqv/post_correction/model/checkpoint/word2num/checkpoint-153441')
device = "cuda:0"
pretrained = pretrained.to(device)


In [13]:
pattern_year_with_month = re.compile(r'((ngày|mùng)\s)?(?(1)((mười|mươi|một|mốt|hai|ba|bốn|tư|năm|lăm|sáu|bảy|bẩy|tám|chín)(\s)?){1,2})(tháng\s)(?(6)((một|hai|ba|tư|bốn|năm|sáu|bảy|bẩy|tám|chín|mười|mười một|mười hai)\s))năm', re.IGNORECASE)

pattern_num = re.compile(r'((không\s?)*(một|hai|ba|bốn|tư|năm|sáu|bảy|bẩy|tám|chín|mười)\s?)((linh|lẻ|không|một|mốt|hai|ba|bốn|tư|năm|lăm|nhăm|sáu|bảy|bẩy|tám|chín|mười|mươi|chục|trăm|nghìn|ngàn|triệu|tỉ|tỷ)\s?)*(?!#)', re.IGNORECASE)

pattern_more1num = re.compile(r'((không\s)*(một|hai|ba|bốn|tư|năm|sáu|bảy|bẩy|tám|chín|mười)\s?)((linh|lẻ|không|một|mốt|hai|ba|bốn|tư|năm|lăm|nhăm|sáu|bảy|bẩy|tám|chín|mười|mươi|chục|trăm|nghìn|ngàn|triệu|tỉ|tỷ)\s?)+', re.IGNORECASE)

pattern_year_only_full_num = re.compile(r'(một|hai|ba|bốn|năm|sáu|bảy|bẩy|tám|chín|mười)\s(mốt|lăm|tư|mươi|linh|lẻ|chục|trăm|nghìn|triệu|tỉ|tỷ)', re.IGNORECASE)

pattern_num_with_phay = re.compile(r'((không\s)*(một|hai|ba|bốn|tư|năm|sáu|bảy|bẩy|tám|chín|mười)\s?)((linh|lẻ|không|một|mốt|hai|ba|bốn|tư|năm|lăm|nhăm|sáu|bảy|bẩy|tám|chín|mười|mươi|chục|trăm|nghìn|ngàn|triệu|tỉ|tỷ)\s?)*,((không|một|hai|ba|bốn|năm|sáu|bảy|bẩy|tám|chín)\s?)+', re.IGNORECASE)


full_num_word_list = ['mươi', 'mười','linh','lẻ','chục','trăm','nghìn','ngàn','triệu','tỉ','tỷ','lăm','nhăm']

def check_single_num(num):
    for word in full_num_word_list:
        if word in num:
            return False
    return True


def num_with_phay_process(match):
    whole_num = match.group(0)
    num_list = whole_num.strip().split(',')
    new_num_list = []
    for num in num_list:
        if check_single_num(num):
            new_num_list.append(str(w2n_single(num)))
        else:
            new_num_list.append(str(w2n(num)))
    return ','.join(new_num_list) + ' '


def num_process(match):
    whole_num = match.group(0)
    year = ''
    if whole_num.startswith('năm'):
        if check_year_only_full_num(whole_num):
            year = '##' + whole_num[:3] + '##' + ' ' 
            whole_num = whole_num[3:]
    # else:
    if check_single_num(whole_num):
        return year + str(w2n_single(whole_num)) + ' '
    else:
        return year + str(w2n(whole_num)) + ' '


def year_with_month_num_process(match):
    num = match.group(0).strip()
    if check_single_num(num):
        return str(w2n_single(num)) + ' '
    else:
        return str(w2n(num)) + ' '

def year_with_month_process(match):
    result = match.group(0)[:-3] + '##' + match.group(0)[-3:] + '##'
    return re.sub(pattern_num, year_with_month_num_process, result)
    

def check_year_only_full_num(s):
    if not s.startswith('năm'):
        # print('doesnt start with nam')
        return False
    remove_nam = s[4:]
    for word in full_num_word_list:
        if remove_nam.startswith(word):
            return False
    result = re.search(pattern_year_only_full_num, s)
    if result:
        return True
    return False


number = re.compile(r'(0|1|2|3|4|5|6|7|7|8|9){2,}')

def add_space_to_num(match):
    str_l = [char for char in match.group(0)]
    return ' '.join(str_l)


def convert_num(sentence):
    sentence = re.sub(pattern_year_with_month, year_with_month_process, sentence)
    sentence = re.sub(pattern_num_with_phay, num_with_phay_process, sentence)
    # print(sentence)
    sentence = re.sub(pattern_more1num, num_process, sentence)
    sentence = re.sub(number, add_space_to_num, sentence)
    sentence = sentence.replace('##', '')
    sentence = ' '.join(sentence.split())
    return sentence

In [14]:
test_text = [
    'năm hai ba',
    'ba,năm triệu',
    'hai không hai mốt',
    'sđt không chín Bảy chín một tám Hai sáu Hai tám alo',
    'ngày hai lăm tháng mười hai năm hai không hai mốt',
    'vào năm hai không hai mốt',
    'hai trăm linh hai',
    'năm trăm lẻ một',
    'hay năm hai không hai mốt',
    'năm nghìn không trăm lẻ bảy',
    'ngày mùng một tháng một năm hai nghìn'
]
for sentence in test_text:
    print(sentence, convert_num(sentence), sep='|')

năm hai ba|5 2 3
ba,năm triệu|3,5 triệu
hai không hai mốt|2 0 2 1
sđt không chín Bảy chín một tám Hai sáu Hai tám alo|sđt 0 9 7 9 1 8 2 6 2 8 alo
ngày hai lăm tháng mười hai năm hai không hai mốt|ngày 2 5 tháng 1 2 năm 2 0 2 1
vào năm hai không hai mốt|vào năm 2 0 2 1
hai trăm linh hai|2 0 2
năm trăm lẻ một|5 0 1
hay năm hai không hai mốt|hay năm 2 0 2 1
năm nghìn không trăm lẻ bảy|5 0 0 7
ngày mùng một tháng một năm hai nghìn|ngày mùng 1 tháng 1 năm 2 0 0 0


In [15]:
sentence = 'ngày mùng một tháng một năm hai nghìn'
start = time.time()

try:
    sentence = convert_num(sentence)
except Exception as e:
    print(str(e))
print(sentence)
# model i choose you!
inputs = tokenizer.encode(sentence, return_tensors='pt')
inputs = inputs.to(device)
outputs = pretrained.generate(inputs, max_length=128, num_beams=6, early_stopping=True)

outputs = outputs.tolist()
print(outputs)
print(tokenizer.batch_decode(outputs))
output_sentence = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(f'model output: {output_sentence}')
print('model time: {}'.format(time.time() - start))

# rule tiem
output_sentence = output_sentence.replace('đô la', '$')
output_sentence = output_sentence.replace(' phẩy ', ',')
output_sentence = output_sentence.replace(' phần trăm', '%')
output_sentence = output_sentence.replace(' a còng', '@')
# try:
#     output_sentence = convert_num(output_sentence)
# except Exception as e:
#     print(str(e))
print(f'input: {sentence}')
print(f'output: {output_sentence}')
print('total time taken: {}'.format(time.time() - start))

ngày mùng 1 tháng 1 năm 2 0 0 0
[[0, 0, 43, 16, 16, 76, 2005, 2005, 2005, 2]]
['<s> <s> ngày một một 2 0 0 0 </s>']
model output: ngày một một 2 0 0 0
model time: 0.16053175926208496
input: ngày mùng 1 tháng 1 năm 2 0 0 0
output: ngày một một 2 0 0 0
total time taken: 0.16091394424438477


In [92]:
from tqdm import tqdm

In [31]:
sentence_list = []
with open('input_ASR.txt') as f:
    for sentence in tqdm(f):
        sentence = sentence.strip()
        inputs = tokenizer.encode(sentence, return_tensors='pt')
        inputs = inputs.to(device)
        outputs = pretrained.generate(inputs, max_length=128, num_beams=6, early_stopping=True)

        outputs = outputs.tolist()
        sentence_list.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


218it [01:35,  3.33it/s]


In [32]:
with open('output_asr.txt', 'w') as new_f:
    for sentence in sentence_list:
        new_f.write(sentence + '\n')


In [4]:
tmp = PhobertTokenizer.from_pretrained('/home3/phungqv/phobert_tokenizer')

In [7]:
tokenizer.encode('khủng_long')

[0, 12118, 2]

In [20]:
tokenizer.decode(2138)

'1 0 0 %'

In [87]:
sentence = 'mình muốn mở mới thẻ thì mình làm như thế nào nhở'

inputs = tmp.encode(sentence, return_tensors='pt')
inputs = inputs.to(device)
outputs = pretrained.generate(inputs, max_length=128, num_beams=6, early_stopping=True)


outputs = outputs.tolist()
print(tmp.batch_decode(outputs))
print(tmp.batch_decode(outputs, skip_special_tokens=True)[0])

['<s> <s> mình muốn mở mới thẻ thì mình làm như thế nào nhở </s>']
mình muốn mở mới thẻ thì mình làm như thế nào nhở
